In [1]:
import os
import requests
import json
from dotenv import load_dotenv
import pandas as pd
import geopandas as gpd
from cartoframes.viz import Map, Layer, popup_element

In [2]:
load_dotenv()

True

In [3]:
key = os.getenv("KEY")

In [35]:
def get_results_from_foursquare (query, location, limit):
    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={location[0]}%2C{location[1]}&sort=DISTANCE&{limit}=5"
    headers = {
        "Accept": "application/json",
        "Authorization": key
    }
    response = requests.get(url, headers=headers)
    return response.json()["results"]

In [36]:
query = "cafe"
Ironhack_lat=41.3977737
Ironhack_lon=2.1882674
location = [Ironhack_lat, Ironhack_lon]
limit = 10
results = get_results_from_foursquare (query, location, limit)

In [37]:
results[0]

{'fsq_id': '506ee46ae4b06a509dceb3d0',
 'categories': [{'id': 13334,
   'name': 'Sandwich Restaurant',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/deli_',
    'suffix': '.png'}}],
 'chains': [],
 'distance': 140,
 'geocodes': {'main': {'latitude': 41.398975, 'longitude': 2.188485},
  'roof': {'latitude': 41.398975, 'longitude': 2.188485}},
 'link': '/v3/places/506ee46ae4b06a509dceb3d0',
 'location': {'address': 'Carrer de Tànger, 22',
  'admin_region': 'Cataluña',
  'country': 'ES',
  'cross_street': '',
  'formatted_address': 'Carrer de Tànger, 22, 08018 Barcelona Catalonia',
  'locality': 'Barcelona',
  'neighborhood': ['El Poblenou'],
  'postcode': '08018',
  'region': 'Catalonia'},
 'name': 'Carrot Café',
 'related_places': {},
 'timezone': 'Europe/Madrid'}